In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/11 mei/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 43
class_id = 103

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 43 class 103.xlsx")


file_to_subject = {}
teacher_tag = '0x24046130BA41'

for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr.'] == tagnr, 'subject ID'].values[0] #had to manually change [tagnr] to [tagnr.] for this day
    except:
        print("Tag not found:" + tag_id)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Mounted at /content/drive
Tag not found:0x24046130BA41


In [2]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-2-3edadb4a2ad4>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-3edadb4a2ad4>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-3edadb4a2ad4>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['

In [3]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
               TimeStamp         X         Y         Z
5727 2023-11-05 11:28:44  0.964741  1.030321  0.632388
5728 2023-11-05 11:28:45  0.964741  1.030321  0.632388
5729 2023-11-05 11:28:46  0.964741  1.030321  0.632388
5730 2023-11-05 11:28:47  0.964741  1.030321  0.632388
5731 2023-11-05 11:28:48  0.401233  1.030321  0.462724
5732 2023-11-05 11:28:49 -0.066237  1.030321  0.613049
5733 2023-11-05 11:28:50  0.229497  1.030321  0.576237
5734 2023-11-05 11:28:51  0.838572  1.030321  0.606133
5735 2023-11-05 11:28:52  1.524427  1.030321  0.630250
5736 2023-11-05 11:28:53  1.524427  1.030321  0.323390
5737 2023-11-05 11:28:54  1.869545  1.030321  0.315575

Maximum value of X:
               TimeStamp         X         Y         Z
7024 2023-11-05 11:50:21  4.579191  1.030321  0.376478
7025 2023-11-05 11:50:22  4.579191  1.030321  0.376478
7026 2023-11-05 11:50:23  4.579191  1.030321  0.376478
7027 2023-11-05 11:50:24  4.579191  1.030321  0.377928
7028 2023-11-05 11:50:25

In [4]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}
teacher_tag = '0x24046130BA41'
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr.'] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_10.0 with Tagnr 30     3.919890  2.426887  0.463924
Person_30.0 with Tagnr 19     6.086348  2.786395  0.495462
Person_16.0 with Tagnr 14     4.027632  3.494596  0.448085
Person_23.0 with Tagnr 32     4.623326  5.581503  0.505349
Person_25.0 with Tagnr 22     7.274220  2.900347  0.556284
Person_7.0 with Tagnr 7       1.436502  2.933677  0.357021
Person_14.0 with Tagnr 11     2.192118  4.630676  0.393991
Person_5.0 with Tagnr 18      4.328108  2.576058  0.449912
Person_18.0 with Tagnr 28     5.864962  4.877031  0.502309
Person_8.0 with Tagnr 29      6.034008  5.306773  0.596987
Person_13.0 with Tagnr 9      3.432325  2.808583  0.349600
Person_6.0 with Tagnr 26      6.571218  3.569198  0.510245
Person_24.0 with Tagnr 21     6.978415  4.675784  0.485060
Person_22.0 with Tagnr 24     5.510216  2.740486  0.457151
Person_21.0 with Tagnr 25     5.868521  3.394402  0.610218
Person_Teacher with Tagnr 35  1.890584  2.448962  0.5375

In [5]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
30  4.301970  5.089374  0.621476
19  5.784646  1.756738  0.670476
14  4.124328  3.486674  0.294314
32  4.461827  4.625587  0.523548
22  6.706267  2.988282  0.490638
7   1.246687  2.382979  0.249257
11  2.239677  1.762205  0.252526
18  4.720352  2.553669  0.413678
28  5.835100  4.471442  0.479013
29  5.987930  4.695806  0.822180
9   2.504860  2.375144  0.226764
26  6.215660  3.471507  0.438077
21  6.939207  1.940492  0.722384
24  5.771172  2.638862  0.317292
25  6.397913  3.548079  0.638101
35  7.214733  3.429708  0.649040
34  3.614884  3.308795  0.361459
6   2.127057  4.650837  0.475677
20  6.415621  1.712951  0.482824
17  4.576006  1.682418  0.502400
13  1.287365  3.226884  0.266035
31  3.394311  4.696261  0.486937
5   2.353696  3.529048  0.299935
27  6.882902  4.379782  0.619229
10  1.672086  1.546443  0.250104
33  3.305710  1.731220  0.432555
1   1.319272  4.408401  0.448857
3   1.855268  3.246760  0.442046
15  3.975554  1.737718  0.383500
2   3.2270

In [6]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
30  4.414104  4.918636  0.820946
19  5.832708  1.832559  0.482488
14  4.142389  3.559466  0.242264
32  4.343886  4.602375  0.467232
22  6.755359  2.915325  0.354292
7   1.260184  2.406157  0.296188
11  2.234350  1.659891  0.221260
18  4.730813  2.348139  0.412423
28  5.604253  4.561623  0.688888
29  5.791792  5.137766  0.957259
9   2.504860  2.375144  0.226764
26  6.119472  3.627717  0.432488
21  7.135175  2.084792  0.710598
24  5.861117  2.797711  0.520617
25  6.435109  3.449482  0.465183
35  7.453627  3.322074  0.597549
34  3.925766  3.658276  0.554498
6   2.121212  4.680883  0.470287
20  6.423745  1.685461  0.487911
17  4.648574  1.642553  0.455147
13  1.195885  3.415704  0.290109
31  3.441236  4.623200  0.460259
5   2.301740  3.345894  0.210842
27  6.872207  4.330161  1.074125
10  1.675105  1.421965  0.332918
33  3.063202  1.489163  0.395209
1   1.319272  4.408401  0.438114
3   1.855268  3.246760  0.451302
15  4.001238  1.769657  0.299529
2   3.0419

In [7]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 35

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

3.3524137967939662 30
2.200903115417754 19
3.090929049440499 14
3.0014358360269364 32
0.6733451073047324 22
6.059142375766767 7
5.247070128206 11
2.6437437333963656 18
1.7287558221734942 28
1.7629658860362134 29
4.826489703337499 9
0.9999461947582197 26
1.5144896965159713 21
1.6459964714990583 24
0.8253515319301448 25
3.6018789628874197 34
5.232169836412391 6
1.8936299566103618 20
3.1647908695354174 17
5.930836528550446 13
4.024894895472388 31
4.862051720114478 5
1.006356370511767 27
5.853855043210123 10
4.26207862224206 33
5.976143950011817 1
5.362586330558395 3
3.654464179164894 15
4.06914684442002 2
The closest tracker to the teacher is 22 with a distance of 0.6733451073047324


In [8]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher

for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within full distance of anyone: 9493
Person 10.0 with Tracker 30: 3463 times within full distance of teacher, category: above average with proximity percentage 36.48
Person 20.0 with Tracker 10: 2073 times within full distance of teacher, category: above average with proximity percentage 21.84
Person 7.0 with Tracker 7: 929 times within full distance of teacher, category: above average with proximity percentage 9.79
Person 9.0 with Tracker 13: 871 times within full distance of teacher, category: above average with proximity percentage 9.18
Person 17.0 with Tracker 3: 645 times within full distance of teacher, category: above average with proximity percentage 6.79
Person 19.0 with Tracker 5: 562 times within full distance of teacher, category: above average with proximity percentage 5.92
Person 14.0 with Tracker 11: 471 times within full distance of teacher, category: above average with proximity percentage 4.96
Person 25.0 with Tracker 22: 165 times within full 

In [9]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 1338
Person 20.0 with Tracker 10: 700 times within half distance of teacher, category: above average with proximity percentage 52.32
Person 17.0 with Tracker 3: 579 times within half distance of teacher, category: above average with proximity percentage 43.27
Person 2.0 with Tracker 6: 27 times within half distance of teacher, category: below average with proximity percentage 2.02
Person 19.0 with Tracker 5: 7 times within half distance of teacher, category: below average with proximity percentage 0.52
Person 13.0 with Tracker 9: 6 times within half distance of teacher, category: below average with proximity percentage 0.45
Person 21.0 with Tracker 25: 6 times within half distance of teacher, category: below average with proximity percentage 0.45
Person 22.0 with Tracker 24: 5 times within half distance of teacher, category: below average with proximity percentage 0.37
Person 25.0 with Tracker 22: 3 times within half distance of t

In [10]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 415
Person 17.0 with Tracker 3: 407 times within quarter distance of teacher, category: above average with proximity percentage 98.07
Person 25.0 with Tracker 22: 3 times within quarter distance of teacher, category: below average with proximity percentage 0.72
Person 13.0 with Tracker 9: 3 times within quarter distance of teacher, category: below average with proximity percentage 0.72
Person 5.0 with Tracker 18: 1 times within quarter distance of teacher, category: below average with proximity percentage 0.24
Person 6.0 with Tracker 26: 1 times within quarter distance of teacher, category: below average with proximity percentage 0.24
Person 10.0 with Tracker 30: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 30.0 with Tracker 19: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 16.0 with Tracker 14: 0 times withi

In [11]:
import pandas as pd


merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31 (1).xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31 (1).xlsx", index=False)